# Cooccurrence Lab

In [9]:
import pandas as pd   
import numpy as np
from IPython.display import display_html 
from scipy.sparse import csc_matrix

from recsys.services.io import IOService
from recsys.data.dataset import Dataset

ModuleNotFoundError: No module named 'scipy'

In [2]:
space = "\xa0" * 10

## Test
The cooccurrene for users 36 and 69 are listed below; however, the test only shows 1, 10, and 208. 

In [3]:
DESTINATION = "tests/testdata/operators/data_operators/cooccurrence/item_ratings_ctr_10pct.pkl"
corating = IOService.read(DESTINATION)
corating[(2,44)]

[20, 29828, 60860, 146193]

In [8]:
filepath = "tests/testdata/operators/data_operators/ratings_user_random_sample_10pct.pkl"
ratings = IOService.read(filepath)
a = ratings[(ratings['movieId'] == 2)]
b = ratings[(ratings['movieId'] == 44)]
c = a[['userId','movieId']].merge(b[['userId','movieId']], on='userId', how='inner')
users = c['userId'].values
d = a[a['userId'].isin(users)]
e = b[b['userId'].isin(users)]
c
#df1_styler = d.style.set_table_attributes("style='display:inline'").set_caption('User 2')
#df2_styler = e.style.set_table_attributes("style='display:inline'").set_caption('User 44')
#display_html(df1_styler._repr_html_()+ space  + df2_styler._repr_html_(), raw=True)

,userId,movieId_x,movieId_y
0,20,2,44
1,907,2,44
2,947,2,44
3,1203,2,44
4,1247,2,44
...,...,...,...
590,160945,2,44
591,160990,2,44
592,161184,2,44
593,162269,2,44


## Vectorize Ids
Reindex user and item ids from 0 to n, then create a binary utility sparse matrix and see if we have enough memory to perform matrix multiplication to reveal related neighbors and items. Using the full dataset... 

In [5]:
filepath = "data/dev/train.pkl"
name = "ratings_dev"
description = "Ratings Dataset (Raw)"
df = IOService.read(filepath=filepath)
df.head()


,userId,movieId,rating,timestamp
16940364,109832,32,4.00,822873600
16940356,109832,17,5.00,822873600
16940359,109832,21,4.00,822873600
16940377,109832,70,4.00,822873600
16940355,109832,12,1.00,822873600


In [6]:
users = np.sort(df['userId'].unique())
dfu = pd.DataFrame(data=users, columns=['userId'])
dfu.reset_index(inplace=True)
dfu = dfu.rename(columns={'index': 'useridx'})
dfu

,useridx,userId
0,0,69
1,1,70
2,2,80
3,3,83
4,4,107
...,...,...
11301,11301,162446
11302,11302,162451
11303,11303,162477
11304,11304,162519


In [21]:
users = [2,4,5,7,8,10,22,15,32,17]
items = [3,9,22,33,6,8,7,10,12,32]
a = np.zeros(40)
b = np.zeros(40)
a[users] = 1
b[items] = 1
print(a)
print(b)
a.T.dot(b)

[0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]


array(1)